In [2]:
from pyscf import gto, scf, dft, fci, mcscf, ao2mo, tools, ci
import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=200)

## RS-DFT purely in pyscf

In [ ]:
mol = gto.M(
    atom = 'C  0.0      0.0      0.66242; \
            C  0.0      0.0     -0.66242; \
            H -0.12018  0.91284  1.23164; \
            H  0.12018 -0.91284  1.23164; \
            H  0.12018  0.91284 -1.23164; \
            H -0.12018 -0.91284 -1.23164;',
    basis="6-31g*",
    verbose=3
)

hf = scf.RHF(mol)

In [ ]:
# 1. run mean-field calculation
hf.kernel()

In [ ]:
# Fock matrix and energy for a given density in AO basis
# for doubly occupied orbitals
def fock(D, h, g):

    print('One-electron energy: ', np.einsum('ij,ij', h, D))
    J = np.einsum('ijkl,kl->ij', g, D)        # Coulomb
    print('Hartree energy: ',  .5*np.einsum('ij,ij', J, D))
    K = -0.5*np.einsum('ilkj,kl->ij', g, D)   # Exchange
    print('Exchange energy: ', .5*np.einsum('ij,ij', K, D))
    F = h + J + K                             # Fock matrix

    E = .5*np.einsum('ij,ij->', h + F, D)

    return F, E

# compute AO to MO transformation of two-electron integrals
# for the given set of MO coefficients C
def eri_ao2mo(C, g):
    pqrw = np.einsum("pqrs,sw->pqrw", g   , C)
    pqvw = np.einsum("pqrw,rv->pqvw", pqrw, C)
    puvw = np.einsum("pqvw,qu->puvw", pqvw, C)
    tuvw = np.einsum("puvw,pt->tuvw", puvw, C)
    return tuvw

def update_total_density(C_act, D_ao_inactive, D_mo_active):
    D_ao_active = np.einsum("pt, qu, tu->pq", C_act, C_act, D_mo_active)
    D_ao_total = D_ao_active + D_ao_inactive
    return D_ao_total


In [ ]:
# manual implementation of CASCI
inactive = [0,1,2,3,4,5]
active = [6,7,8,9]

nel = 4
nmo = len(active)

# get MO coefficients
C = hf.mo_coeff
C_occ = C[:, inactive]
C_act = C[:, active]

# inactive density matrix in AO basis
D_I = 2.0*np.einsum('ik,jk->ij', C_occ, C_occ)

# get nuclear repulsion energy
E_nuc = hf.mol.get_enuc()
print("Nuclear repulsion energy: ", E_nuc)

# get one- and two-body integrals in AO basis
h = hf.get_hcore()
g = mol.intor('int2e')

# get inactive Fock matrix in AO basis and inactive energy
F_I, E_I = fock(D_I, h, g)

# transform Fock matrix to active MO basis
F_mo = np.einsum("pq, qu, pt->tu", F_I, C_act, C_act)
print("Inactive Foock matrix in active MO basis:\n", F_mo)

# ao to mo transformation
g_mo = eri_ao2mo(C_act, g)
# print(g_mo)

# create and solve the FCI problem
fci_solver = fci.direct_spin0.FCI()
fci_solver.nroots = 2
fci_solver.verbose = 5
E_A, CI = fci_solver.kernel(h1e=F_mo, eri=g_mo, norb=nmo, nelec=nel)
# print(fci.spin_square(CI, norb=nmo, nelec=nel))

# print(fci_solver.make_rdm12(CI, norb=len(active), nelec=4))

# total energy
print(E_I, E_nuc, E_A)
print(E_I + E_A + E_nuc)

# double check with pyscf implementation
# cas = mcscf.CASCI(hf, 2, 4)
# cas.kernel()

In [ ]:
def print_ci(fci_solver, thresh = 0.0):
    nel = fci_solver.nelec
    norb = fci_solver.norb
    occslst = fci.cistring._gen_occslst(range(norb), nel//2)
    pad = 4 + norb

    for root in range(fci_solver.nroots):
        print(f'Root {root}:')
        if fci_solver.nroots == 1:
            ci_vector = fci_solver.ci
        else:
            ci_vector = fci_solver.ci[root]
        print(f'  {"Conf": <{pad}} CI coefficients')
        for i,occsa in enumerate(occslst):
            for j,occsb in enumerate(occslst):
                if abs(ci_vector[i,j]) < thresh:
                    continue
                # generate the CI string and print it
                occ = ''
                for k in range(norb):
                    if k in occsa and k in occsb:
                        occ += '2'
                    elif k in occsa and k not in occsb:
                        occ += 'u'
                    elif k not in occsa and k in occsb:
                        occ += 'd'
                    else:
                        occ += '0'
                print('  %s     %+.8f'%(occ, ci_vector[i,j]))

In [ ]:
print_ci(fci_solver, thresh=1e-6)

In [ ]:
# omega = 0.0001
omega = 0.5
# omega = 10000
mf = dft.RKS(mol)
mf.chkfile = None
mf._numint.libxc = dft.xcfun
mf.xc = f'LDAERF + LR_HF({omega})' # why do we need to add LR_HF(0.5) here?
# mf.xc = 'LDAERF'
mf.omega = omega
# mf.xc = 'HF'

In [ ]:
# run the sr-dft calculation
mf.kernel()

## Manual implementation of RS-DFT

In the following I implement manually RS-DFT, and I test it by first calculating the mean-field energy manually and comparing it to the one obtained from pyscf. Then I break the MO space into inactive and active, and I try again to obtain the same mean-field energy by taking a trivial active space of $2n$ electrons in $n$ orbitals.

In [ ]:
print('\n######## RHF-srDFT energy ########')

# get the density matrix in AO basis
D = mf.make_rdm1()

# get nuclear repulsion energy
E_n = mf.mol.get_enuc()
print('Nuclear repulsion energy:', E_n)

########### ONE-BODY INTEGRALS ###########
# get one-body integrals in AO basis
h = mf.get_hcore()
E_h = np.einsum('ij,ij', h, D)
print('One-electron energy:', E_h)

########### TWO-BODY INTEGRALS ###########
# get the full range two-body AO integrals
g = mol.intor('int2e')
# get lr two-body AO integrals
with mol.with_range_coulomb(omega= omega):
    g_lr = mol.intor('int2e')
# get sr two-body AO integrals
with mol.with_range_coulomb(omega=-omega):
    g_sr = mol.intor('int2e')
assert(np.allclose(g, g_lr + g_sr))

########### COULOMB/HARTREE MATRIX ###########
# get the Hartree/Coulomb matrix in AO basis
J    = mf.get_j(dm=D) # this is full range
J_lr = mf.get_j(dm=D, omega= omega) # this is long-range
J_sr = mf.get_j(dm=D, omega=-omega) # this is short-range
assert(np.allclose(J, J_lr + J_sr))
# we can also manually compute the Hartree/Coulomb matrix
# using the two-body integrals
_J    = np.einsum('ijkl,kl->ij', g   , D)
_J_lr = np.einsum('ijkl,kl->ij', g_lr, D)
_J_sr = np.einsum('ijkl,kl->ij', g_sr, D)
assert(np.allclose(J, _J))
assert(np.allclose(J_lr, _J_lr))
assert(np.allclose(J_sr, _J_sr))
# Hartree energy
E_J = .5*np.einsum('ij,ij', J, D)
print('Hartree energy:', E_J)

########### EXCHANGE MATRIX ###########
# same business for the exchange matrix
K    = mf.get_k(dm=D) # this is full range
K_lr = mf.get_k(dm=D, omega= omega) # this is long-range
K_sr = mf.get_k(dm=D, omega=-omega) # this is short-range
assert(np.allclose(K, K_lr + K_sr))
# we can also manually compute the Hartree/Coulomb matrix
_K    = np.einsum('ikjl,kl->ij', g   , D)
_K_lr = np.einsum('ikjl,kl->ij', g_lr, D)
_K_sr = np.einsum('ikjl,kl->ij', g_sr, D)
assert(np.allclose(K, _K))
assert(np.allclose(K_lr, _K_lr))
assert(np.allclose(K_sr, _K_sr))
# in RS-DFT we are only including the long-range exchange
E_K_lr = -.25*np.einsum('ij,ij', K_lr, D)
print('LR Exchange energy:', E_K_lr)

### Notice that J and K can be computed using a single function call
_J_lr_, _K_lr_ = mf.get_jk(dm=D, omega=omega)
assert(np.allclose(_J_lr_, J_lr))
assert(np.allclose(_K_lr_, K_lr))

########### Vxc MATRIX and Exc ###########
# get the exchange and correlation matrix and energy
n, E_xc, V_xc = mf._numint.nr_rks(mol, mf.grids, mf.xc, D)
# the exchange energy cannot be calculated by tracing V_xc and D
# because E_xc depends non-linearly on the density matrix
print('Difference between Exc and {Vxc,D}:', E_xc - np.einsum('ij,ij', V_xc, D))
print('Exchange-correlation energy:', E_xc)
# we can now build the KS matrix

########### KS/FOCK MATRIX ###########
# first we build the "pure" Fock matrix part
F = h + J - .5*K_lr
# note that if this would have been a normal or hybrid DFT
# calculation, then we would have used the full range exchange K
E_F = .5*np.einsum('ij,ij', h + F, D)
print('Fock energy:', E_F)
assert(np.allclose(E_F, E_h + E_J + E_K_lr))

# if we add the V_xc to the Fock matrix, we get the KS matrix
F_ks = F + V_xc
# we can get the effective potential of the KS matrix from the
# RKS object as well
V_eff = mf.get_veff(dm=D)
# this is like F_ks without the one-electron term
assert(np.allclose(F_ks - h, V_eff))

# and we can also get the KS matrix from the RKS object
_F_ks = mf.get_fock(dm=D)
assert(np.allclose(F_ks, _F_ks))
# if this would be a HF calculation, then mf.get_fock() would
# would give back just the Fock matrix (no V_xc) and mf.get_veff()
# would give back only the HF effective potential V_hf

########### TOTAL ENERGY ###########
E_rs = E_h + E_J + E_K_lr + E_xc + E_n
print("RHF-srDFT energy:", E_rs)
print("Difference with mf.e_tot:", E_rs - mf.e_tot)

# we can also get the total energy by tracing the Fock matrix
# with the density matrix and then adding the exchange-correlation
# energy and the nuclear repulsion energy
assert(np.allclose(E_rs, E_F + E_xc + E_n))

Now I try to break the MO space into inactive and active, and set up already the scheme with the FCI solver for the LR active space part (even though it is a trivial AS). I should be able to get the same energy as the RHF-srDFT.

To see CAS-srDFT as an embedding problem, it helps to visualize all the energy quantities needed and divided them by range and MO space
| range       | inactive       | active                |
|-------------|----------------|-----------------------|
| no range    | $E_h$          | $E_h$                 |
| short range | $E_J + E_{xc}$ | $E_J + E_{xc}$        |
| long range  | $E_J + E_K$    | $E_J + E_K + E_{cas}$ |


In [ ]:
print('\n######## CAS-srDFT energy ########')

# define inactive and active orbital spaces
inactive = [0,1,2,3,4]
active = [5,6,7]
n_ina = len(inactive)
n_act = len(active)
n_el = 6

# set occupations for inactive and active MOs
mo_occ_inactive = np.zeros_like(mf.mo_occ)
mo_occ_inactive[inactive] = mf.mo_occ[inactive]
mo_occ_active = mf.mo_occ - mo_occ_inactive

# get MO coefficients
C = mf.mo_coeff
C_I = C[:, inactive]
C_A = C[:, active]
# inactive charge density (P in szabo) in AO basis
# normally called density matrix, or  1-RDM in AO basis
D_I  = 2.0*np.einsum('ip,jp->ij', C_I, C_I)
_D_I = mf.make_rdm1(C, mo_occ_inactive) # same as above
assert(np.allclose(D_I, _D_I))
# active charge density/density matrix
D_A_mo = np.zeros((n_act,n_act))  # manually form the 1-RDM in MO basis
D_A_mo[0,0] = 2.0
D_A_mo[1,1] = 2.0
D_A_mo[2,2] = 2.0
# transform it to AO basis
D_A  = np.einsum('tu,pt,qu->pq', D_A_mo, C_A, C_A)
_D_A = mf.make_rdm1(C, mo_occ_active) # same as above
assert(np.allclose(D_A, _D_A))
# total charge density/density matrix in AO basis
D = D_I + D_A
assert(np.allclose(D, mf.make_rdm1()))

# get one--body integrals in AO basis
h = mf.get_hcore()

########### COULOMB/HARTREE MATRIX ###########
# get all the coulomb integrals
J   = mf.get_j(dm=D)
J_I = mf.get_j(dm=D_I)
J_A = mf.get_j(dm=D_A)

# notice that E_H[rho + drho] /= E_H[rho] + E_H[drho]
# it is not a linear functional!
E_J   = 0.5*np.einsum('ij,ij', J  , D)
E_J_I = 0.5*np.einsum('ij,ij', J_I, D_I)
E_J_A = 0.5*np.einsum('ij,ij', J_A, D_A)
print('Non-linearity of Hartree functional:', E_J - E_J_I - E_J_A)
# non-linear coupling terms
E_J_IA = 0.5*np.einsum('ij,ij', J_I, D_A)
E_J_AI = 0.5*np.einsum('ij,ij', J_A, D_I)
assert(np.allclose(E_J, E_J_I + E_J_A + E_J_IA + E_J_AI))
# E_J_IA is the Coulomb energy felt by the active electrons
# due to the inactive electrons, while E_J_AI the opposite
# These two terms are equal, since the Coulumb repulsion energy
# between two charge distributions is symmetric. It is easy to
# see with the equations: E_J_IA = sum_pqrs D_I[pq] (pq|rs) D_A[rs]
# and E_J_AI = sum_pqrs D_A[pq] (pq|rs) D_I[rs] and it is clear that
# whether we contract first pq or rs, we get the same result, since
# (pq|rs) = (rs|pq)
assert(np.allclose(E_J_IA, E_J_AI))

# now we also need to introduce the range separation
J_lr   = mf.get_j(dm=D,   omega=omega)
J_I_lr = mf.get_j(dm=D_I, omega=omega)
J_A_lr = mf.get_j(dm=D_A, omega=omega)
assert(np.allclose(J_lr, J_I_lr + J_A_lr))
J_sr   = mf.get_j(dm=D,   omega=-omega)
J_I_sr = mf.get_j(dm=D_I, omega=-omega)
J_A_sr = mf.get_j(dm=D_A, omega=-omega)
assert(np.allclose(J_sr, J_I_sr + J_A_sr))

E_J_lr    = 0.5*np.einsum('ij,ij', J_lr  , D)
E_J_I_lr  = 0.5*np.einsum('ij,ij', J_I_lr, D_I)
E_J_A_lr  = 0.5*np.einsum('ij,ij', J_A_lr, D_A)
E_J_IA_lr = 0.5*np.einsum('ij,ij', J_I_lr, D_A)
E_J_AI_lr = 0.5*np.einsum('ij,ij', J_A_lr, D_I)
assert(np.allclose(E_J_lr, E_J_I_lr + E_J_A_lr + E_J_IA_lr + E_J_AI_lr))

E_J_sr    = 0.5*np.einsum('ij,ij', J_sr  , D)
E_J_I_sr  = 0.5*np.einsum('ij,ij', J_I_sr, D_I)
E_J_A_sr  = 0.5*np.einsum('ij,ij', J_A_sr, D_A)
E_J_IA_sr = 0.5*np.einsum('ij,ij', J_I_sr, D_A)
E_J_AI_sr = 0.5*np.einsum('ij,ij', J_A_sr, D_I)
assert(np.allclose(E_J_sr, E_J_I_sr + E_J_A_sr + E_J_IA_sr + E_J_AI_sr))
assert(np.allclose(E_J, E_J_sr + E_J_lr))

########### EXCHANGE MATRIX ###########
# get all the exchange integrals (only long-range)
K_lr   = mf.get_k(dm=D,   omega=omega)
K_I_lr = mf.get_k(dm=D_I, omega=omega)
K_A_lr = mf.get_k(dm=D_A, omega=omega)

# the exchange energy functional is also non-linear
# with respect to the density matrix
E_K    = -0.25*np.einsum('ij,ij', K_lr  , D)
E_K_I  = -0.25*np.einsum('ij,ij', K_I_lr, D_I)
E_K_A  = -0.25*np.einsum('ij,ij', K_A_lr, D_A)
E_K_IA = -0.25*np.einsum('ij,ij', K_I_lr, D_A)
E_K_AI = -0.25*np.einsum('ij,ij', K_A_lr, D_I)
assert(np.allclose(E_K, E_K_I + E_K_A + E_K_IA + E_K_AI))

########### Vxc MATRIX and Exc ###########
# get the exchange and correlation integrals (only short-range)
n, E_xc  , V_xc   = mf._numint.nr_rks(mol, mf.grids, mf.xc, D)
n, E_xc_I, V_xc_I = mf._numint.nr_rks(mol, mf.grids, mf.xc, D_I)
n, E_xc_A, V_xc_A = mf._numint.nr_rks(mol, mf.grids, mf.xc, D_A)
# note that while the Hartree and exact exchange potentials are
# linear wrt the density, and only their energy functionals are
# non-linear, the xc potential is also non-linear in the density!
assert(not np.allclose(V_xc, V_xc_I + V_xc_A))
print('Non-linearity of E_xc:', E_xc - (E_xc_I + E_xc_A))

########### CAS-srDFT POTENTIAL and ENERGY ###########
# to perform CAS-srDFT we need three ingredients:
# 1. the LR inactive Fock operator and its energy
# 2. the SR embedding potential and energy
# 3. the embedding potential for the LR active part

#####
# 1. the LR inactive Fock operator and its energy are easy
F_I_lr = h + J_I_lr - .5*K_I_lr                     # eq. 9 in Rossmannek et al. with 0.5 factor
E_I_lr = .5*np.einsum('ij,ij', h + F_I_lr, D_I)     # eq. 4 in Rossmannek et al.

# the inactive LR energy is essentially the LR HF energy of the inactive part
print('LR inactive energy:', E_I_lr)
# note that F_I_lr is part of the embedding potential for point 3.

#####
# 2. the SR energy is a bit more complicated
# here we need to compute the SR Hxc energy for all electrons
# in principle, I would think that this is simply E_xc + E_J_sr
# but looking at eq. 37 in Hedegard et al., this is not the case
# here we try to compute E_fix^SA, which is the same as the 2nd
# and 3rd line of eq. 15 in Rossmannek et al.
E_I_sr = (E_J_I_sr - E_J_A_sr) + (E_xc - np.einsum('tu,tu', V_xc, D_A))
# what surprises me is that we only use E_J_I_sr and not the full E_J_sr
# this actually comes from the linearization of the method, which seems
# required...

# the full range inactive energy is then
E_I = E_I_lr + E_I_sr
print('SR+LR inactive energy:', E_I)

# the SR embedding potential is for all the electrons
V_emb_sr = J_I_sr + J_A_sr + V_xc
# note that I break J_sr = J_I_sr + J_A_sr, to highlight that
# J_I_sr does not change becasue it depends only on the inactive density
# while J_A_sr changes because it depends on the active density

####
# 3. the embedding potential for the LR active part
# here we want to put together the embedding potential for the LR active
# part and solve the FCI problem

# the LR active part is embedded in the LR inactive part and in the full SR part
V_emb = F_I_lr + V_emb_sr
# but this is almost equal to the KS matrix we get with mf.get_fock()
_V_emb = mf.get_fock(dm=D) - (J_A_lr - .5*K_A_lr)
assert(np.allclose(V_emb, _V_emb))

# transform embedding potential to active MO basis
V_emb_mo = np.einsum("pq,qu,pt->tu", V_emb, C_A, C_A)

# get lr two-body AO integrals
with mol.with_range_coulomb(omega= omega):
    g_lr = mol.intor('int2e') # there have no symmetry
    _g_lr_mo = mol.ao2mo(C_A) # these come with 4-fold symmetry
# my own ao to mo transformation of the LR two-electron integrals
g_lr_mo = eri_ao2mo(C_A, g_lr)
assert(np.allclose(g_lr_mo, ao2mo.addons.restore(1,_g_lr_mo, n_act)))
# or also with the ao2mo module
assert(np.allclose(ao2mo.kernel(g_lr, C_A), g_lr_mo))

# create and solve the FCI problem
fci_solver = fci.direct_spin0.FCI()
fci_solver.nroots = 1
fci_solver.verbose = 5
# solve the FCI problem with the embedding potential and lr ERI
E_A_lr, CI_vec = fci_solver.kernel(h1e=V_emb_mo, eri=_g_lr_mo, norb=n_act, nelec=n_el)
print("LR active energy:", E_A_lr)
print('<S^2> =', fci.spin_square(CI_vec, norb=n_act, nelec=n_el)[0])

# get nuclear repulsion energy
E_n = mf.mol.get_enuc()
print("Nuclear repulsion energy:", E_n)

########### TOTAL ENERGY ###########
E_tot = E_I + E_A_lr + E_n
print("CAS-srDFT energy:", E_tot)
print("Difference with mf.e_tot:", E_tot - mf.e_tot)

In [ ]:
def max_fci_srdft(mf, active, n_el, max_iter=100, threshold=1e-6):

    # some settings
    dm_history = []
    e_history = []
    omega = mf.omega
    # print('omega:', omega)
    spin = mf.mol.spin
    # print('spin:', spin)

    # define inactive and active orbital spaces
    inactive = np.where(mf.mo_occ != 0)
    inactive = [i for i in inactive[0] if i not in active]
    print('Inactive orbitals:', inactive)
    print('Active orbitals:', active)
    n_ina = len(inactive)
    n_act = len(active)

    # set occupations for inactive and active MOs
    mo_occ_inactive = np.zeros_like(mf.mo_occ)
    mo_occ_inactive[inactive] = mf.mo_occ[inactive]
    mo_occ_active = mf.mo_occ - mo_occ_inactive

    # get MO coefficients
    C = mf.mo_coeff
    C_I = C[:, inactive]
    C_A = C[:, active]
    # inactive charge density (P in szabo) in AO basis
    # normally called density matrix, or  1-RDM in AO basis
    D_I = mf.make_rdm1(C, mo_occ_inactive)

    # active charge density/density matrix
    D_A_mo = np.zeros((n_act,n_act))  # manually form the 1-RDM in MO basis
    for i in range(np.count_nonzero(mo_occ_active)):
        D_A_mo[i,i] = 2.0

    # transform it to AO basis
    D_A  = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
    # D_A = mf.make_rdm1(C, mo_occ_active) # same as above
    dm_history.append(D_A_mo)
    # total charge density/density matrix in AO basis
    D = mf.make_rdm1()
    assert(np.allclose(D, D_I + D_A))

    ########### FCI-srDFT POTENTIAL and ENERGY ###########
    # to perform FCI-srDFT we need three ingredients:
    # 1. the LR inactive Fock operator and its energy
    # 2. the SR embedding potential and energy
    # 3. the embedding potential for the LR active part

    V_emb = mf.get_fock()

    # transform embedding potential to active MO basis
    # V_emb_mo = np.einsum("pq,qu,pt->tu", V_emb, C_A, C_A)

    # get lr two-body AO integrals
    with mf.mol.with_range_coulomb(omega= omega):
        g_lr_mo = mf.mol.ao2mo(C_A) # these come with 4-fold symmetry

    # get nuclear repulsion energy
    E_n = mf.mol.get_enuc()

    # initialize inactive energy
    E_I = mf.e_tot - E_n

    # create the FCI solver
    fci_solver = fci.direct_spin1.FCI()
    fci_solver = fci.addons.fix_spin_(fci_solver, ss=spin)

    n_iter = 0
    delta_e = float('NaN')
    E_A = 0.0
    S2 = mf.spin_square()[0]
    e_history.append(mf.e_tot)
    converged = False
    # start the iterative loop
    print("Iter\tConvergence\tInactive energy\tActive energy\tTotal energy\t<Ψ|S^2|Ψ>")
    # print(f"{n_iter}\t{delta_e:.8f}\t{E_I + E_n:.8f}\t{E_A:.8f}\t{e_history[-1]:.8f}\t{S2:.2f}")
    while n_iter < max_iter:
        n_iter += 1

        J_A_lr, K_A_lr = mf.get_jk(dm=D_A, omega=omega)
        active_fock_operator = J_A_lr - 0.5*K_A_lr
        inactive_fock_operator = V_emb - active_fock_operator

        # transform embedding potential to active MO basis
        V_emb_mo = np.einsum("pq,qu,pt->tu", inactive_fock_operator, C_A, C_A)

        E_I -= np.einsum('pq,pq->', D_A, V_emb)
        E_I += .5*np.einsum('pq,pq->', D_A, active_fock_operator)

        # solve the FCI problem with the embedding potential and lr ERI
        E_A, CI_vec = fci_solver.kernel(h1e=V_emb_mo, eri=g_lr_mo, norb=n_act, nelec=n_el)
        # print("Active energy:", E_A)
        S2, mult = fci.spin_square(CI_vec, norb=n_act, nelec=n_el)

        # new active density matrix
        D_A_mo = fci_solver.make_rdm1(CI_vec, norb=n_act, nelec=n_el)
        dm_history.append(D_A_mo)
        # print('Active-space RDM:\n', D_A_mo)
        # transform it to AO basis
        D_A = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
        # check convergence
        e_history.append(E_I + E_A + E_n)
        delta_e = e_history[-1] - e_history[-2]
        # print(f"{n_iter}\t{delta_e:.8f}\t{e_history[-1]:.8f}\t{S2:.2f}")
        print(f"{n_iter}\t{delta_e:>11.8f}\t{E_I + E_n:.8f}\t{E_A:.8f}\t{e_history[-1]:.8f}\t{S2:.2f}")

        converged = np.abs(delta_e) < threshold
        if converged:
            print("Converged!")
            return e_history[-1], dm_history[-1]

        # update density matrix
        D = D_A + D_I

        # update inactive energy
        E_I = mf.energy_tot(dm=D) - E_n
        # print("Inactive energy:", E_I + E_n)


In [ ]:
def fci_srdft(mf, active, n_el, max_iter=100, threshold=1e-6):

    # some settings
    dm_history = []
    e_history = []
    omega = mf.omega
    # print('omega:', omega)
    spin = mf.mol.spin
    # print('spin:', spin)

    # define inactive and active orbital spaces
    inactive = np.where(mf.mo_occ != 0)
    inactive = [i for i in inactive[0] if i not in active]
    print('Inactive orbitals:', inactive)
    print('Active orbitals:', active)
    n_ina = len(inactive)
    n_act = len(active)

    # set occupations for inactive and active MOs
    mo_occ_inactive = np.zeros_like(mf.mo_occ)
    mo_occ_inactive[inactive] = mf.mo_occ[inactive]
    mo_occ_active = mf.mo_occ - mo_occ_inactive
    # print('Active MO occupations:', mo_occ_active)

    # get MO coefficients
    C = mf.mo_coeff
    C_I = C[:, inactive]
    C_A = C[:, active]
    # inactive charge density (P in szabo) in AO basis
    # normally called density matrix, or  1-RDM in AO basis
    D_I = mf.make_rdm1(C, mo_occ_inactive)

    # active charge density/density matrix
    D_A_mo = np.zeros((n_act,n_act))  # manually form the 1-RDM in MO basis
    for i in range(np.count_nonzero(mo_occ_active)):
        D_A_mo[i,i] = 2.0

    # transform it to AO basis
    D_A  = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
    # D_A = mf.make_rdm1(C, mo_occ_active) # same as above
    dm_history.append(D_A_mo)
    # total charge density/density matrix in AO basis
    D = mf.make_rdm1()
    assert(np.allclose(D, D_I + D_A))

    ########### FCI-srDFT POTENTIAL and ENERGY ###########
    # to perform FCI-srDFT we need three ingredients:
    # 1. the LR inactive Fock operator and its energy
    # 2. the SR embedding potential and energy
    # 3. the embedding potential for the LR active part

    # get one-body integrals in AO basis
    h = mf.get_hcore()

    #####
    # 1. the LR inactive Fock operator and its energy are easy
    J_I_lr, K_I_lr = mf.get_jk(dm=D_I, omega=omega)
    F_I_lr = h + J_I_lr - .5*K_I_lr                     # eq. 9 in Rossmannek et al. with 0.5 factor
    E_I_lr = .5*np.einsum('ij,ij', h + F_I_lr, D_I)     # eq. 4 in Rossmannek et al.

    # 2. the SR embedding potential and energy
    n, E_xc, V_xc = mf._numint.nr_rks(mf.mol, mf.grids, mf.xc, D)
    J_I_sr = mf.get_j(dm=D_I, omega=-omega)
    J_A_sr = mf.get_j(dm=D_A, omega=-omega)
    V_emb_sr = J_I_sr + J_A_sr + V_xc               # second-to-last terms in eq. 15 in Rossmannek et al.
    E_J_I_sr  = 0.5*np.einsum('ij,ij', J_I_sr, D_I)
    E_J_A_sr  = 0.5*np.einsum('ij,ij', J_A_sr, D_A)
    # second and third lines in eq. 15 in Rossmannek et al.
    E_I_sr = (E_J_I_sr - E_J_A_sr) + (E_xc - np.einsum('tu,tu', V_xc, D_A))

    # the full range inactive energy is then
    E_I = E_I_lr + E_I_sr

    V_emb = F_I_lr + V_emb_sr
    J_A_lr, K_A_lr = mf.get_jk(dm=D_A, omega=omega)
    _V_emb = mf.get_fock(dm=D) - (J_A_lr - .5*K_A_lr)
    assert(np.allclose(V_emb, _V_emb))

    # transform embedding potential to active MO basis
    V_emb_mo = np.einsum("pq,qu,pt->tu", V_emb, C_A, C_A)

    # get lr two-body AO integrals
    with mf.mol.with_range_coulomb(omega= omega):
        g_lr_mo = mf.mol.ao2mo(C_A) # these come with 4-fold symmetry

    # get nuclear repulsion energy
    E_n = mf.mol.get_enuc()
    # print("Inactive energy:", E_I + E_n)

    # create the FCI problem
    fci_solver = fci.direct_spin1.FCI()
    fci_solver = fci.addons.fix_spin_(fci_solver, ss=spin)

    n_iter = 0
    delta_e = float('NaN')
    E_A = 0.0
    S2 = mf.spin_square()[0]
    e_history.append(mf.e_tot)
    converged = False
    # start the iterative loop
    print("Iter\tConvergence\tInactive energy\tActive energy\tTotal energy\t<Ψ|S^2|Ψ>")
    # print(f"{n_iter}\t{delta_e:.8f}\t{E_I + E_n:.8f}\t{E_A:.8f}\t{e_history[-1]:.8f}\t{S2:.2f}")
    while n_iter < max_iter:
        n_iter += 1

        # solve the FCI problem with the embedding potential and lr ERI
        E_A, CI_vec = fci_solver.kernel(h1e=V_emb_mo, eri=g_lr_mo, norb=n_act, nelec=n_el)
        S2, mult = fci.spin_square(CI_vec, norb=n_act, nelec=n_el)

        # new active density matrix
        D_A_mo = fci_solver.make_rdm1(CI_vec, norb=n_act, nelec=n_el)
        dm_history.append(D_A_mo)
        # print('Active-space RDM:\n', D_A_mo)
        # transform it to AO basis
        D_A = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
        # check convergence
        e_history.append(E_I + E_A + E_n)
        delta_e = e_history[-1] - e_history[-2]
        print(f"{n_iter}\t{delta_e:>11.8f}\t{E_I + E_n:.8f}\t{E_A:.8f}\t{e_history[-1]:.8f}\t{S2:.2f}")

        converged = np.abs(delta_e) < threshold
        if converged:
            print("Converged!")
            return e_history[-1], dm_history[-1]

        # update density matrix
        D = D_A + D_I
        # print('AS energy:', e_history[-1])
        # print('MF energy:', mf.energy_tot(dm=D))

        # get new embedding potential
        n, E_xc, V_xc = mf._numint.nr_rks(mf.mol, mf.grids, mf.xc, D)
        J_A_sr = mf.get_j(dm=D_A, omega=-omega)
        V_emb_sr = J_I_sr + J_A_sr + V_xc
        E_J_A_sr  = 0.5*np.einsum('ij,ij', J_A_sr, D_A)
        E_I_sr = (E_J_I_sr - E_J_A_sr) + (E_xc - np.einsum('tu,tu', V_xc, D_A))
        # the full range inactive energy is then
        E_I = E_I_lr + E_I_sr
        # print("Inactive energy:", E_I + E_n)
        V_emb = F_I_lr + V_emb_sr
        J_A_lr, K_A_lr = mf.get_jk(dm=D_A, omega=omega)
        _V_emb = mf.get_fock(dm=D) - (J_A_lr - .5*K_A_lr)
        assert(np.allclose(V_emb, _V_emb))
        # transform embedding potential to active MO basis
        V_emb_mo = np.einsum("pq,qu,pt->tu", V_emb, C_A, C_A)


In [ ]:
R = 3.0

n2 = gto.M(
    atom = f'N 0.0 0.0 0.0; \
             N 0.0 0.0 {R};',
    basis="sto-3g",
    spin=0,
    verbose=3
)

omega = 7.0
mf = dft.RKS(n2)
mf._numint.libxc = dft.xcfun
mf.xc = f'LDAERF + LR_HF({omega})'
mf.omega = omega

In [ ]:
mf.kernel()
tools.molden.dump_scf(mf, 'n2.molden')

In [ ]:
active = [4,5,6,7,8,9]
n_el = 6
max_iter = 100
e,d = fci_srdft(mf, active, n_el, max_iter)

In [ ]:
e,d = max_fci_srdft(mf, active, n_el, 100)

In [ ]:
energies = []
densities = []
# energies_fci = []
Rn = [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0]

for R in Rn:
    print(f'R = {R}')
    n2 = gto.M(
        atom = f'N 0.0 0.0 0.0; \
                 N 0.0 0.0 {R};',
        basis='sto-3g',
        spin=0,
    )

    omega = 7.0
    mf = dft.RKS(n2)
    mf._numint.libxc = dft.xcfun
    mf.xc = f'LDAERF + LR_HF({omega})'
    mf.omega = omega
    mf.kernel()
    active = [4,5,6,7,8,9]
    n_el = 6
    max_iter = 70
    e, d = max_fci_srdft(mf, active, n_el, max_iter, threshold=1e-4)
    energies.append(e)
    densities.append(d)

    # hf = scf.RHF(n2)
    # hf.kernel()
    # e_fci = fci.FCI(hf).kernel()[0]
    # energies_fci.append(e_fci)

In [ ]:
energies_4in5 = energies

In [ ]:
energies_8in7 = energies

In [ ]:
import matplotlib.pyplot as plt

plt.plot(Rn, energies, 'o-')
# plt.plot(Rn, energies_4in5, 'o-')
# plt.plot(Rn, energies_8in7, 'o-')
plt.plot(Rn, energies_fci, 'o-')

In [ ]:
def fast_fci_srdft(mf, active, n_el, max_iter=100, threshold=1e-6):

    # some settings
    D_A_history = []
    E_history = []
    omega = None
    if hasattr(mf, 'omega'):
        omega = mf.omega
    spin = mf.mol.spin

    # define inactive and active orbital spaces
    inactive = np.where(mf.mo_occ != 0)
    inactive = [i for i in inactive[0] if i not in active]
    print('Inactive orbitals:', inactive)
    print('Active orbitals:', active)
    n_act = len(active)

    # set occupations for inactive and active MOs
    mo_occ_inactive = np.zeros_like(mf.mo_occ)
    mo_occ_inactive[inactive] = mf.mo_occ[inactive]
    mo_occ_active = mf.mo_occ - mo_occ_inactive

    # get MO coefficients
    C = mf.mo_coeff
    C_A = C[:, active]
    # inactive density matrix in AO basis
    D_I = mf.make_rdm1(C, mo_occ_inactive)

    # in the initial step, the active density matrix is equal to the
    # HF density matrix within the active subspace
    D_A_mo = np.zeros((n_act,n_act))  # manually form the 1-RDM in MO basis
    for i in range(np.count_nonzero(mo_occ_active)):
        D_A_mo[i,i] = 2.0

    # transform 1-RDM to AO basis
    D_A = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
    # D_A = mf.make_rdm1(C, mo_occ_active) # same as above
    D_A_history.append(D_A_mo)
    # total density matrix in AO basis
    D = D_I + D_A

    # get lr two-body MO integrals
    with mf.mol.with_range_coulomb(omega=omega):
        g_lr_mo = mf.mol.ao2mo(C_A)

    # get nuclear repulsion energy
    E_n = mf.mol.get_enuc()

    # create the FCI solver
    fci_solver = fci.direct_spin1.FCI()
    fci_solver = fci.addons.fix_spin_(fci_solver, ss=spin)

    n_iter = 0
    E_history.append(mf.e_tot)
    converged = False
    # start the iterative loop
    print("Iter\tConvergence\tInactive energy\tActive energy\tTotal energy\t<Ψ|S^2|Ψ>")
    while n_iter < max_iter:
        n_iter += 1

        # get the KS potential and LR active components to subtract
        F_ks = mf.get_fock(dm=D)   # this is ks_mat
        J_A_lr, K_A_lr = mf.get_jk(dm=D_A, omega=omega)

        V_emb = F_ks - (J_A_lr - 0.5*K_A_lr)  # J_A_lr - 0.5*K_A_lr is ks_ref

        # transform embedding potential to active MO basis
        V_emb_mo = np.einsum("pt,pq,qu->tu", C_A, V_emb, C_A)

        # compute inactive energy
        E_I  =    mf.energy_tot(dm=D) - E_n
        E_I -=    np.einsum('pq,pq->', D_A, F_ks)
        E_I += .5*np.einsum('pq,pq->', D_A, (J_A_lr - 0.5*K_A_lr)) # this is eeri

        # solve the FCI problem with the embedding potential and lr ERI
        E_A, CI_vec = fci_solver.kernel(h1e=V_emb_mo, eri=g_lr_mo, norb=n_act, nelec=n_el)
        S2, mult = fci.spin_square(CI_vec, norb=n_act, nelec=n_el)

        # new active density matrix
        D_A_mo = fci_solver.make_rdm1(CI_vec, norb=n_act, nelec=n_el)
        D_A_history.append(D_A_mo)
        # transform it to AO basis
        D_A = np.einsum('pt,tu,qu->pq', C_A, D_A_mo, C_A)
        # check convergence
        E_history.append(E_I + E_A + E_n)
        delta_e = E_history[-1] - E_history[-2]
        print(f"{n_iter}\t{delta_e:>11.8f}\t{E_I + E_n:.8f}\t{E_A:.8f}\t{E_history[-1]:.8f}\t{S2:.2f}")

        converged = np.abs(delta_e) < threshold
        if converged:
            print("Converged!")
            return E_history[-1], D_A_history[-1]

        # update density matrix
        D = D_I + D_A


In [ ]:
c2h4 = gto.M(
    atom = 'C  0.0      0.0      0.66242; \
            C  0.0      0.0     -0.66242; \
            H -0.12018  0.91284  1.23164; \
            H  0.12018 -0.91284  1.23164; \
            H  0.12018  0.91284 -1.23164; \
            H -0.12018 -0.91284 -1.23164;',
    basis="sto-3g",
    spin=0,
    verbose=3
)

omega = 0.5
mf = dft.RKS(c2h4)
mf.chkfile = None
mf._numint.libxc = dft.xcfun
mf.xc = f'LDAERF + LR_HF({omega})' # why do we need to add LR_HF(0.5) here?
mf.omega = omega

hf = scf.RHF(c2h4)

In [ ]:
mf.kernel()
hf.kernel()

In [ ]:
active = [4,5,6,7,8,9]
n_el = 6
max_iter = 100

e,d = fast_fci_srdft(mf, active, n_el, max_iter)

In [ ]:
e,d = fast_fci_srdft(mf, active, n_el, max_iter)

In [ ]:
e_dft_pyscf = mf.e_tot
e_dft_cp2k  = -76.93313472420846

delta_e_dft = e_dft_cp2k - e_dft_pyscf

print('Delta E (DFT) = ', delta_e_dft)

e_cas_pyscf = e + delta_e_dft
e_cas_cp2k  = -76.939250151776108

print('FCI-srDFT energy (PySCF) = ', e_cas_pyscf)
print('FCI-srDFT energy (CP2K)  = ', e_cas_cp2k)
print('Delta E (FCI-srDFT) = ', e_cas_cp2k - e_cas_pyscf)

In [ ]:
e_rhf_pyscf = hf.e_tot
e_rhf_cp2k  = -77.06849289125047

delta_e = e_rhf_cp2k - e_rhf_pyscf

print('Delta E (RHF) = ', delta_e)

e_cas_pyscf = e + delta_e
e_cas_cp2k  = -77.1148781209

print('FCI-srDFT energy (PySCF) = ', e_cas_pyscf)
print('FCI-srDFT energy (CP2K)  = ', e_cas_cp2k)
print('Delta E (FCI-srDFT) = ', e_cas_cp2k - e_cas_pyscf)

In [ ]:
c2h4 = gto.M(
    atom = 'C  0.0      0.0      0.66242; \
            C  0.0      0.0     -0.66242; \
            H -0.12018  0.91284  1.23164; \
            H  0.12018 -0.91284  1.23164; \
            H  0.12018  0.91284 -1.23164; \
            H -0.12018 -0.91284 -1.23164;',
    basis="3-21g",
    spin=0,
    verbose=5
)

hf = scf.RHF(c2h4)

In [ ]:
hf.kernel()

In [ ]:
active = [4,5,6,7,8,9]
n_el = 6
max_iter = 100

e,d = (mf, active, n_el, max_iter)

In [7]:
def log_ci_states(ci_solver, thresh=1e-6):
    norb = ci_solver.norb
    nel_a, nel_b = ci_solver.nelec
    fci_occslst_a = fci.cistring.gen_occslst(range(norb), nel_a)
    fci_occslst_b = fci.cistring.gen_occslst(range(norb), nel_b)
    fci_strs_a = fci.cistring.make_strings(range(norb), nel_a)
    fci_strs_b = fci.cistring.make_strings(range(norb), nel_b)
    fci_space = len(fci_occslst_a) * len(fci_occslst_b)

    for root in range(ci_solver.nroots):
        if ci_solver.nroots == 1:
            ci_vector = ci_solver.ci
        else:
            ci_vector = ci_solver.ci[root]

        print(
            f"Logging CI vectors and coefficients > {thresh} for root number {root}:"
        )

        # decimate the occs lists to get only selected determinants
        if isinstance(ci_solver, fci.SCI):
            strs_a = np.intersect1d(ci_vector._strs[0], fci_strs_a, assume_unique=True)
            strs_b = np.intersect1d(ci_vector._strs[1], fci_strs_b, assume_unique=True)
            idx_a = fci.cistring.strs2addr(norb, nel_a, strs_a)
            idx_b = fci.cistring.strs2addr(norb, nel_b, strs_b)
            occslst_a = fci_occslst_a[idx_a]
            occslst_b = fci_occslst_b[idx_b]
            sci_space = len(occslst_a) * len(occslst_b)
            print(f"The SCI space contains {sci_space} determinants over {fci_space} ({sci_space/fci_space:.2%})")
        else:
            occslst_a = fci_occslst_a
            occslst_b = fci_occslst_b
            print(f"The FCI space contains {fci_space} determinants.")

        pad = 4 + norb
        coeffs_shown = 0
        print(f'  {"Conf": <{pad}} CI coefficients')
        for i, occsa in enumerate(occslst_a):
            for j, occsb in enumerate(occslst_b):
                if abs(ci_vector[i, j]) < thresh:
                    continue
                # generate the CI string and log it
                occ = ""
                for k in range(norb):
                    if k in occsa and k in occsb:
                        occ += "2"
                    elif k in occsa and k not in occsb:
                        occ += "u"
                    elif k not in occsa and k in occsb:
                        occ += "d"
                    else:
                        occ += "0"
                print("  %s     %+.8f" % (occ, ci_vector[i, j]))
                coeffs_shown += 1

        print(f"There are {coeffs_shown} CI coefficients > {thresh}\n")


In [8]:
#### trying out FCI vs SCI
mol = gto.Mole()
mol.atom = "H 0 0 0; H 0 0 1.2; H 0 0 2.4; H 0 0 3.6"
mol.unit='angstrom'
mol.basis = "6-31g**"
mol.build()

# Run RHF
mf = scf.RHF(mol)
mf.kernel()


converged SCF energy = -2.10815860070824


np.float64(-2.1081586007082436)

In [10]:
# Rotate the Hamiltonians
norb = mol.nao
print("Number of orbitals:", norb)
nelec = mol.nelectron
print("Number of electrons:", nelec)
mo_coeff = mf.mo_coeff
h1e = mf.get_hcore()
eri = mf._eri
h1e_mo = mo_coeff.T @ h1e @ mo_coeff
eri_mo = ao2mo.kernel(eri, mo_coeff, compact=False).reshape(norb, norb, norb, norb)
e_nuc = mf.energy_nuc()

# Run SCI
scisolver = fci.SCI()
scisolver = fci.selected_ci_spin0.SCI()
scisolver.max_cycle = 100
scisolver.nroots = 1
scisolver.verbose = 4

# convergence is when the SCI space does not increase anymore
# or when the ground state energy change is lower than tol*1e3
# (this means that excited state energies may not be converged)
scisolver.conv_tol = 1e-9

# these two are impoortant for logging the CI coefficients
scisolver.norb = norb
scisolver.nelec = (nelec//2, nelec//2)

# minimum coeff. for determinants used to enlarge the space by
# applying singles and double excitation operators to them
# default is 5e-3
scisolver.ci_coeff_cutoff = .5e-3
# cutoff to include a determinant in the enlarged space
# the selection is the coefficient of the reference determinant
# times the relevant ERI for the single or double excitation
# default is 5e-3
scisolver.select_cutoff = .5e-3
e_sci, scivec = scisolver.kernel(h1e_mo, eri_mo, norb, nelec)

# important to manually store the CI coefficients
scisolver.ci = scivec

print("Selected CI energy: {}".format(e_sci + e_nuc))

# Give some information about the CI vector
occslst_a = fci.cistring.gen_occslst(range(norb), nelec//2)
n_civec_a = len(occslst_a)
print("Full CI space size:", n_civec_a**2)

Number of orbitals: 20
Number of electrons: 4
cycle 0  E = -4.1081198068737  dE = -4.1081198
cycle 1  E = -4.10905179548415  dE = -0.00093198861
Selected CI  E = -4.10905752575484
Selected CI energy: -2.198139819654843
Full CI space size: 36100
Logging CI vectors and coefficients > 0.01 for root number 0:
The SCI space contains 16129 determinants over 36100 (44.68%)
  Conf                     CI coefficients
  22000000000000000000     +0.95199274
  u2d00000000000000000     +0.01136387
  2u0d0000000000000000     -0.01410337
  uudd0000000000000000     +0.03677794
  u200d000000000000000     -0.01445082
  2u000d00000000000000     -0.01793461
  2u00000d000000000000     +0.01216545
  20200000000000000000     -0.19972916
  udud0000000000000000     -0.04579373
  20u0d000000000000000     -0.02476145
  udu00d00000000000000     -0.02788757
  20u000d0000000000000     +0.03306259
  udu0000d000000000000     -0.01335973
  d2u00000000000000000     +0.01136387
  02200000000000000000     -0.07284165
  d

In [12]:

log_ci_states(scisolver, 5e-2)

Logging CI vectors and coefficients > 0.05 for root number 0:
The SCI space contains 16129 determinants over 36100 (44.68%)
  Conf                     CI coefficients
  22000000000000000000     +0.95199274
  20200000000000000000     -0.19972916
  02200000000000000000     -0.07284165
  duud0000000000000000     -0.08248580
  uddu0000000000000000     -0.08248580
  02020000000000000000     -0.05397063
There are 6 CI coefficients > 0.05



In [6]:
int(np.ceil(np.sqrt(80)))

9

In [5]:
np.sqrt(80.25)

np.float64(8.958236433584458)

In [238]:
def filter_ci_wf(ci_vector, nelec, norb, thresh=1e-6):
    if isinstance(nelec, tuple):
        nel_a, nel_b = nelec
    else:
        nel_a = nel_b = nelec//2
    fci_strs_a = fci.cistring.make_strings(range(norb), nel_a)
    fci_strs_b = fci.cistring.make_strings(range(norb), nel_b)

    strs_a = []
    strs_b = []
    idx_a = []
    idx_b = []
    for i, str_a in enumerate(fci_strs_a):
        for j, str_b in enumerate(fci_strs_b):
            # include strings with CI coefficients above threshold
            if abs(ci_vector[i, j]) >= thresh:
                idx_a.append(i)
                idx_b.append(j)
                strs_a.append(str_a)
                strs_b.append(str_b)

    idxs = np.intersect1d(idx_a, idx_b)
    strs = np.intersect1d(strs_a, strs_b)
    return ci_vector[idxs,:][:,idxs], (np.asarray(strs), np.asarray(strs))


In [239]:
myci = ci.CISD(mf)
ecisd, cisdvec = myci.kernel()
cisdvec_as_fci = myci.to_fcivec(cisdvec)
cisdvec_as_fci

E(RCISD) = -2.194662677547618  E_corr = -0.08650407683937768


array([[ 0.957041, -0.      ,  0.011579, ..., -0.      , -0.      , -0.000586],
       [-0.      , -0.189988, -0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.011579, -0.      , -0.068628, ...,  0.      ,  0.      ,  0.      ],
       ...,
       [-0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [-0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [-0.000586,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [240]:
nel = (nelec//2, nelec//2)

In [253]:
nroots = 5
nsci = int(np.ceil(nroots/2))
if nsci > norb-nel[0]+1 or nsci > norb-nel[1]+1:
    raise RuntimeError('Not enough selected-CI space for %d states' % nroots)

ci_strs_a = np.zeros(nsci, dtype=int)
ci_strs_b = np.zeros(nsci, dtype=int)

core_a = '1'*(nel[0]-1)
core_b = '1'*(nel[1]-1)

ci0 = np.zeros((nsci, nsci))

for i in range(nsci):
    ci_strs_a[i] = int(bin(2**i) + core_a, 2)
    ci_strs_b[i] = int(bin(2**i) + core_b,2)
    ci0[i,i] = np.exp(-1.0 * -np.log(0.01)*i/nsci)

ci_strs = (ci_strs_a, ci_strs_b)
sci0 = fci.selected_ci._as_SCIvector(ci0, ci_strs)

In [194]:
ci0, ci_strs = filter_ci_wf(cisdvec_as_fci, nelec, norb, thresh=5e-3)
sci0 = fci.selected_ci._as_SCIvector(ci0, ci_strs)

In [248]:
sci0._strs

(array([3, 5, 9]), array([3, 5, 9]))

In [254]:
scisolver.kernel(h1e_mo, eri_mo, norb, nelec, sci0)

cycle 0  E = -4.03793087173983  dE = -4.0379309


cycle 1  E = -4.10891531676377  dE = -0.070984445
cycle 2  E = -4.109086058881  dE = -0.00017074212
Selected CI  E = -4.10908629988644


(np.float64(-4.109086299886438),
 SCIvector([[ 0.951975, -0.      ,  0.011377, ...,  0.000169,  0.000395, -0.000611],
            [-0.      , -0.199743, -0.      , ..., -0.      , -0.      ,  0.      ],
            [ 0.011377, -0.      , -0.072832, ...,  0.000117, -0.000413,  0.000162],
            ...,
            [ 0.000169, -0.      ,  0.000117, ...,  0.000048,  0.000005, -0.000011],
            [ 0.000395, -0.      , -0.000413, ...,  0.000005,  0.000042, -0.000007],
            [-0.000611,  0.      ,  0.000162, ..., -0.000011, -0.000007,  0.000019]]))

In [250]:
scivec

SCIvector([[ 0.951976, -0.      ,  0.011375, ...,  0.000169,  0.000395, -0.000611],
           [-0.      , -0.199743, -0.      , ..., -0.      , -0.      ,  0.      ],
           [ 0.011375, -0.      , -0.072832, ...,  0.000117, -0.000413,  0.000162],
           ...,
           [ 0.000169, -0.      ,  0.000117, ...,  0.000048,  0.000005, -0.000011],
           [ 0.000395, -0.      , -0.000413, ...,  0.000005,  0.000042, -0.000007],
           [-0.000611,  0.      ,  0.000162, ..., -0.000011, -0.000007,  0.000019]])

In [5]:
log_ci_states(scisolver, thresh=5e-2)

Logging CI vectors and coefficients > 0.05 for root number 0:
  Conf         CI coefficients
  22000000     -0.98286028
  20200000     +0.16934708
  02200000     +0.06812991
The SCI space contains 9 determinants over 784 (1.15%)
There are 3 CI coefficients > 0.05

Logging CI vectors and coefficients > 0.05 for root number 1:
  Conf         CI coefficients
  2ud00000     +0.69625323
  2du00000     -0.69625323
  ud200000     -0.12341573
  du200000     +0.12341573
The SCI space contains 9 determinants over 784 (1.15%)
There are 4 CI coefficients > 0.05



In [255]:
# Compared to FCI
fcisolver = fci.FCI(mf)
fcisolver.nroots = 1
e_fci, fcivec = fcisolver.kernel()
print("Difference compared to FCI: {}".format(e_fci - (e_sci+e_nuc)))

Difference compared to FCI: -2.4418640480305953e-05


In [258]:
fcisolver.nelec

(2, 2)

In [23]:
fcivec

FCIvector([[ 0.93971 ,  0.      ,  0.00319 , -0.00381 , -0.      ,  0.038056],
           [ 0.      , -0.243298, -0.      ,  0.      , -0.075406,  0.      ],
           [ 0.00319 , -0.      , -0.077971, -0.113462, -0.      ,  0.006559],
           [-0.00381 ,  0.      , -0.113462, -0.055962,  0.      , -0.007672],
           [-0.      , -0.075406, -0.      ,  0.      , -0.080041,  0.      ],
           [ 0.038056,  0.      ,  0.006559, -0.007672, -0.      ,  0.04318 ]])

In [22]:
e_sci+e_nuc

array([-2.123919, -1.98776 ])

In [24]:
e_fci

array([-2.181861, -2.061636])

In [ ]:
log_ci_states(fcisolver, thresh=1e-2)

In [ ]:
umf = scf.UHF(mol)
umf.kernel()

# Rotate the Hamiltonians
norb = mol.nao
nelec = mol.nelectron
mo_coeff = umf.mo_coeff
h1e = umf.get_hcore()
eri = umf._eri

h1e_mo = np.zeros((2, norb, norb))
h1e_mo[0] = mo_coeff[0].T @ h1e @ mo_coeff[0]
h1e_mo[1] = mo_coeff[1].T @ h1e @ mo_coeff[1]

eri_mo = np.zeros((3, norb, norb, norb, norb))
eri_mo[0] = ao2mo.kernel(eri, mo_coeff[0], compact=False).reshape(norb, norb, norb, norb)
eri_mo[1] = ao2mo.kernel(eri, mo_coeff[0], mo_coeff[0], mo_coeff[1], mo_coeff[1], compact=False).reshape(norb, norb, norb, norb)
eri_mo[2] = ao2mo.kernel(eri, mo_coeff[1], compact=False).reshape(norb, norb, norb, norb)

e_nuc = mf.energy_nuc()

# Run SCI
scisolver = fci.SCI()
scisolver.max_cycle = 100
scisolver.conv_tol = 1e-8
scisolver.nroots = 1
e, civec = scisolver.kernel(h1e_mo, eri_mo, norb, (2,2))
e_sci = e + e_nuc # add nuclear energy
print("Selected CI energy: {}".format(e_sci))

In [ ]:
#### trying out CCSD
from pyscf import cc

mol = gto.Mole()
mol.atom = "H 0 0 0; H 0 0 1.2; H 0 0 2.4; H 0 0 3.6"
mol.unit='angstrom'
mol.basis = "631g"
mol.build()

# Run RHF
mf = scf.RHF(mol)
mf.kernel()

# Run CC
cc_solver = cc.CCSD(mf)
cc_solver.kernel()
e_cc = cc_solver.e_tot
print("CCSD energy: {}".format(e_cc))

# Compared to FCI
fcisolver = fci.FCI(mf)
e_fci, fcivec = fcisolver.kernel()
print("Difference compared to FCI: {}".format(e_fci - e_cc))